### 1. ElasticSearch 연동 및 확인

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [2]:
es = Elasticsearch('http://13.209.84.139:9200')
es.info()

{'name': 'Wilshere_Elasticsearch',
 'cluster_name': 'Wilshere_ES',
 'cluster_uuid': 'e8ZntsY7R7CEZnj8Odv-Rw',
 'version': {'number': '7.12.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '3186837139b9c6b6d23c3200870651f10d3343b7',
  'build_date': '2021-04-20T20:56:39.040728659Z',
  'build_snapshot': False,
  'lucene_version': '8.8.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### 2. ElasticSearch 에 데이터 적재 및 색인

##### 2-1). ElasticSearch 에 index 생성

In [3]:
# 데이터를 색인화하기 위한 함수
def indexing(es, index_name):
    # 이미 존재할 경우 삭제하고 다시 만들기
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    # 인덱스 생성
    print(es.indices.create(index=index_name))

In [11]:
# 인덱스명을 정하기 (자유롭게)
index_name = 'en_wiki_5'
#index_name = 'temp1'
indexing(es, index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'en_wiki_5'}


In [ ]:
# 인덱스 생성 : en_wiki_1~5

##### 2-2_. ElasticSearch 에 데이터 적재(인덱싱)

In [ ]:
./en_wiki_dump/enwiki-20210520_1_result/AA~AD || AA/AB/AC/AD
./en_wiki_dump/enwiki-20210520_2_result/AA~AF || AA/AB/AC/AD/AE/AF
./en_wiki_dump/enwiki-20210520_3_result/AA~AE || AA/AB/AC/AD/AE
./en_wiki_dump/enwiki-20210520_4_result/AA~AF || AA/AB/AC/AD/AE/AF
./en_wiki_dump/enwiki-20210520_5_result/AA~AF || AA/AB/AC/AD/AE/AF
여기까지

In [17]:
import os
import xml.etree.ElementTree as ET
import json, codecs
import xmltodict
from collections import OrderedDict

path = './en_wiki_dump/enwiki-20210520_5_result/AF/'
file_list = os.listdir(path)

for i in range(len(file_list)):
    fr = open(path + file_list[i],'r', encoding="utf-8")
    data = fr.readlines()
    
    for j in range(len(data)):
        es.index(index=index_name, doc_type='string', body=data[j])

    fr.close()
es.indices.refresh(index=index_name)

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

### 3. Elasticsearch 데이터 검색

In [4]:
index_name = "en_wiki*"

In [5]:
results = es.search(index=index_name, body={'from':0, 'size':5, 'query':{'match':{'text':'Kenya'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

score: 11.89215 source: {'id': '145569', 'revid': '39662122', 'url': 'https://en.wikipedia.org/wiki?curid=145569', 'title': 'Kenya Airways', 'text': 'Kenya Airways Ltd., more commonly known as Kenya Airways, is the flag carrier airline of Kenya. The company was founded in 1977, after the dissolution of East African Airways. Its head office is located in Embakasi, Nairobi, with its hub at Jomo Kenyatta International Airport.\nThe airline was owned by the Government of Kenya until , and it was privatised in 1996, becoming the first African flag carrier to successfully do so. Kenya Airways is currently a public-private partnership. The largest shareholder is the Government of Kenya (48.9%), 38.1% is owned by KQ Lenders Company 2017 Ltd. (in turn owned by a consortium of banks), followed by KLM, which has a 7.8% stake in the company. The rest of the shares are held by private owners; shares are traded on the Nairobi Stock Exchange, the Dar es Salaam Stock Exchange, and the Uganda Securitie

### 4. 질의 응답 시스템 구축

In [6]:
# 로드하는 항목들
import argparse
import math
import os
import random
import datasets
import numpy as np
import torch
import collections
import json
import logging
import transformers
import easydict
import pandas as pd

from tqdm import tqdm_notebook
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets, load_metric
from torch.utils.data.dataloader import DataLoader
from typing import Optional, Tuple
from accelerate import Accelerator

# Huggingface transformers의 토크나이저, 설정파일, 최적화 함수 등 사용
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

from utils_qa import postprocess_qa_predictions
from transformers.utils import check_min_version

# huggingface dataset 추가
from datasets import load_dataset

In [7]:
## 명령행 인터페이스, 사전 훈련된 모델을 로드하여 훈련하고 평가하는데까지 필요한 인자를 사용자 정의 인자로 설정하는 내용을 담고 있음.
## 사용법: python example.py --dataset_name squad.json .... 
## 주피터 환경을 위한 인자 설정법 
import easydict
import string
import re
def easydict_args():
    
    args = easydict.EasyDict({
        
        "dataset_name": 'squad',
        "dataset_config_name": 'plain_text',
        "train_file": None,        
        "preprocessing_num_workers": 4,
        "validation_file": 'final_test_sample.json',
        "test_file": 'final_test_no_context.json',
        "test_save_path" : "result/final_test.csv",
        "max_seq_length": 384,
        "pad_to_max_length": None,
        "model_name_or_path": 'pytorch_model.bin',
        "config_name": 'bert-base-cased',
        "tokenizer_name": 'bert-base-cased',
        "use_slow_tokenizer": None,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 3e-5,
        "weight_decay": 0.0,
        "num_train_epochs": 3,
        "max_train_steps": None,
        "gradient_accumulation_steps": 1,
        "lr_scheduler_type": 'linear',
        "num_warmup_steps":0,
        "output_dir": './result/',
        "seed": 42,
        "doc_stride": 128,
        "n_best_size": 20,
        "null_score_diff_threshold": 0.0,
        "version_2_with_negative": False,
        "max_answer_length": 30,
        "max_train_samples": None,
        "max_eval_samples": None,
        "overwrite_cache": False,
        "max_predict_samples": '2000',
    })

    ## 무결성 체크
    ## 인자로 전달한 데이터셋 명이 잘못되었거나 확장자 오류가 있을 경우 raise 이하의 메세지를 출력
    if (
        args.dataset_name is None
        and args.train_file is None
        and args.validation_file is None
        and args.test_file is None
    ):
        raise ValueError("Need either a dataset name or a training/validation/test file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if args.test_file is not None:
            extension = args.test_file.split(".")[-1]
            assert extension in ["csv", "json"], "`test_file` should be a csv or a json file."

    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
    
    ## 위에서 인자로 전달받은 값을 args 변수에 담아 반환
    return args

In [8]:
print(easydict_args())

{'dataset_name': 'squad', 'dataset_config_name': 'plain_text', 'train_file': None, 'preprocessing_num_workers': 4, 'validation_file': 'final_test_sample.json', 'test_file': 'final_test_no_context.json', 'test_save_path': 'result/final_test.csv', 'max_seq_length': 384, 'pad_to_max_length': None, 'model_name_or_path': 'pytorch_model.bin', 'config_name': 'bert-base-cased', 'tokenizer_name': 'bert-base-cased', 'use_slow_tokenizer': None, 'per_device_train_batch_size': 4, 'per_device_eval_batch_size': 4, 'learning_rate': 3e-05, 'weight_decay': 0.0, 'num_train_epochs': 3, 'max_train_steps': None, 'gradient_accumulation_steps': 1, 'lr_scheduler_type': 'linear', 'num_warmup_steps': 0, 'output_dir': './result/', 'seed': 42, 'doc_stride': 128, 'n_best_size': 20, 'null_score_diff_threshold': 0.0, 'version_2_with_negative': False, 'max_answer_length': 30, 'max_train_samples': None, 'max_eval_samples': None, 'overwrite_cache': False, 'max_predict_samples': '2000'}


In [14]:
print(datasets)

raw_datasets = load_dataset('squad', 'plain_text')
print(raw_datasets)
print(type(raw_datasets))

<module 'datasets' from 'c:\\users\\donghwan lee\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\datasets\\__init__.py'>


Reusing dataset squad (C:\Users\Donghwan Lee\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 10570
    })
})
<class 'datasets.dataset_dict.DatasetDict'>


In [9]:
# SQuAD 평가를 위한 전처리 함수
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [10]:
logger = logging.getLogger(__name__)

def main():
    args = easydict_args()

    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    ## 가속화 모드를 지원하며, 해당 모드로 병목이 발생하는 메서드 부분을 대체해서 사용가능.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format = "%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt = "%m/%d/%Y %H:%M:%S",
        level = logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # If passed along, set the training seed now.
    if args.seed is not None:
        set_seed(args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    
    ## Huggingface의 datasets에 존재하는 데이터셋 명을 사용하는 경우에는 저장된 형식에 맞추어서 로드 (위의 링크에서 리스트 참조)
    
    
    
    if args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)

        
    ## 사용자 정의의 데이터셋을 사용하는 경우에는 아래의 조건문 실행. 
    else:
        data_files = {}
        print(args)
        
        if args.train_file is not None:
            data_files["train"] = args.train_file
            #print(data_files["train"])
            
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
            #print(data_files["validation"])
            
        if args.test_file is not None:
            data_files["test"] = args.test_file
            #print(data_files["test"])
            
        extension = args.test_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, field="data")
        raw_datasets = load_dataset(extension, data_files=data_files, field="data")


    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    ## 지정한 모델의 설정에 맞추어서 환경 구성 요소 로드
    ## ex) BERT --> BERTConfig...
    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path)
    else:
        config = CONFIG_MAPPING[args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")
    
    ## 지정한 모델의 설정에 맞추어서 토크나이저 로드
    ## ex) BERT --> BERTTokenizer ... 30,000개의 vocab와 word piece tokenzier를 곁들인...
    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=True)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    ## 지정한 모델 로드하기.
    ## 여기서는 사전 훈련된 모델 상단에 질의응답 TASK를 위한 레이어를 추가하고 미세조정훈련을 하기위해 ForQuestionAnswering을 불러온다.
    if args.model_name_or_path:
        model = AutoModelForQuestionAnswering.from_pretrained(
            args.model_name_or_path,
            from_tf = bool(".ckpt" in args.model_name_or_path),
            config = config,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForQuestionAnswering.from_config(config)

    # Preprocessing the datasets.
    # Preprocessing is slighlty different for training and evaluation.
    
    ## 전처리하기 
#    column_names = raw_datasets["test"].column_names
    column_names = raw_datasets["train"].column_names
    print(column_names)
    
    
    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "context" if "context" in column_names else column_names[1]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]
    
    
    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"

    ## 인자로 전달한 최대 입력 시퀀스 길이가 모델의 최대 입력 임베딩 길이보다 길 경우 에러 반환
    if args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )

    max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)


    # prediction preprocessing
    def prepare_validation_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation = "only_second" if pad_on_right else "only_first",
            max_length = max_seq_length,
            stride = args.doc_stride,
            return_overflowing_tokens = True,
            return_offsets_mapping = True,
            padding = "max_length" if args.pad_to_max_length else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples

    

    #################
    ## 사용자 정의 검증 데이터를 입력 인자로 사용하는 경우, --do_predict을 입력 인자로 반드시 사용해야함.

    if "train" not in raw_datasets:
        raise ValueError("--do_predict requires a test dataset")
    #eval_examples = raw_datasets["test"]

    
    if args.max_eval_samples is not None:
        # We will select sample from whole data
        eval_examples = eval_examples.select(range(args.max_eval_samples))
    # Validation Feature Creation
    eval_dataset = eval_examples.map(
        prepare_validation_features,
        batched = True,
        num_proc = args.preprocessing_num_workers,
        remove_columns = column_names,
        load_from_cache_file = not args.overwrite_cache,
    )

    if args.max_eval_samples is not None:
        # During Feature creation dataset samples might increase, we will select required samples again
        eval_dataset = eval_dataset.select(range(args.max_eval_samples))



    #################
    ## 입력 인코딩을 완료한 데이터셋에 대해서 batch_size 만큼 로드하기 (pytorch 프레임워크의 장점)
    # DataLoaders creation:
    if args.pad_to_max_length:
        # If padding was already done ot max length, we use the default data collator that will just convert everything
        # to tensors.
        data_collator = default_data_collator
    else:
        # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
        # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
        # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

    ## 검증 데이터셋에 대한 로더
    eval_dataset_for_model = eval_dataset.remove_columns(["example_id", "offset_mapping"])
    eval_dataloader = DataLoader(
        eval_dataset_for_model, collate_fn = data_collator, batch_size = args.per_device_eval_batch_size
    )


    ## huggingface에서 제공하는 간편하게 평가 결과를 반환하도록 하는 함수 
    ## 깃헙에서 다운로드 받은 utils_qa.py를 참조
    # Post-processing:
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples = examples,
            features = features,
            predictions = predictions,
            version_2_with_negative = args.version_2_with_negative,
            n_best_size = args.n_best_size,
            max_answer_length = args.max_answer_length,
            null_score_diff_threshold = args.null_score_diff_threshold,
            output_dir = args.output_dir,
            prefix = stage,
        )
        # Format the result to the format the metric expects.
        if args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            #formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
            formatted_predictions = [{"id": normalize_answer(k), "prediction_text": normalize_answer(v)} for k, v in predictions.items()]
            df_pred = pd.DataFrame(formatted_predictions)
            df_pred.to_csv(args.test_save_path , index=False, encoding='utf-8')

        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        return EvalPrediction(predictions = formatted_predictions, label_ids = references)

    ## F1 score, Exact match
    metric = load_metric("squad_v2" if args.version_2_with_negative else "squad")

    #################
    # Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
    def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
        """
        Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
        Args:
            start_or_end_logits(:obj:`tensor`):
                This is the output predictions of the model. We can only enter either start or end logits.
            eval_dataset: Evaluation dataset
            max_len(:obj:`int`):
                The maximum length of the output tensor. ( See the model.eval() part for more details )
        """

        step = 0
        # create a numpy array and fill it with -100.
        ## pad에 해당하는 logit 값을 -100으로 지정
        ## 효율적인 배치 계산을 위한 방법으로 지정한 max sequence length에 메모리 효율을 올리는 방법
        logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
        # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
        for i, output_logit in enumerate(start_or_end_logits):  # populate columns
            # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
            # And after every iteration we have to change the step

            batch_size = output_logit.shape[0]
            cols = output_logit.shape[1]

            if step + batch_size < len(dataset):
                logits_concat[step : step + batch_size, :cols] = output_logit
            else:
                logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

            step += batch_size

        return logits_concat

    # Prediction
    all_start_logits = []
    all_end_logits = []
    for step, batch in enumerate(tqdm_notebook(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            start_logits = outputs.start_logits # 모든 토큰 중에서 가장 probability가 높은 시작점
            end_logits = outputs.end_logits# 모든 토큰 중에서 가장 probability가 높은 끝점
                
            if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor

    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)
    prediction = post_processing_function(eval_examples, eval_dataset, outputs_numpy)
    eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
    logger.info(f"Evaluation metrics: {eval_metric}")


In [ ]:
if __name__ == '__main__':
    main()